# Q1 - Calculate Space Needed With Current Rack Sizes



## Calculations
### Step 1 - Dictionaries and Supporting Data

## Step 2 - Dictionary

In [2]:
# Define the constants & dictionary 
import pandas as pd
import math


# Load the provided Excel data for SKU demand and carton dimensions
file_path = 'Warehouse rack design data (3).xlsx'
warehouse_data = pd.ExcelFile(file_path)

# Parse data
material_demand = warehouse_data.parse('Material Demand')

# Extract SKU-specific parameters
cartons_per_layer = dict(zip(material_demand['Raw Material No.'], material_demand['Cartons per pallet layer']))
layer_height = dict(zip(material_demand['Raw Material No.'], material_demand['Height of one layer']))
demand_8h = dict(zip(material_demand['Raw Material No.'], material_demand['8h']))

print (cartons_per_layer)
print (layer_height)
print (demand_8h)

# Constants
usable_rack_height = 1200  # Usable height for storing raw materials (mm)
total_shelves = 70         # Maximum number of shelves
racks_per_shelf = 5        # Number of racks per shelf
pallets_per_rack = 2       # Number of pallets per rack
total_racks = total_shelves * racks_per_shelf  # Total racks in warehouse
total_pallets = total_racks * pallets_per_rack  # Maximum number of pallets

# Helper: Calculate number of pallets required for each SKU
def calculate_pallets(demand, cartons_per_layer, layer_height, usable_rack_height):
    if layer_height > usable_rack_height:
        raise ValueError(f"Layer height {layer_height} exceeds usable rack height {usable_rack_height}.")
    
    # Total layers needed
    total_layers = math.ceil(demand / cartons_per_layer)
    
    # Max layers per pallet
    max_layers_per_pallet = usable_rack_height // layer_height
    if max_layers_per_pallet <= 0:
        raise ValueError("Invalid configuration: No layers can fit on a pallet due to excessive layer height.")
    
    # Total pallets required
    pallets = math.ceil(total_layers / max_layers_per_pallet)
    return pallets, total_layers  # Returning total layers for additional checks

# Calculate pallets and total layers required for 5-hour demand
pallets_8h = {
    sku: calculate_pallets(demand_8h[sku], cartons_per_layer[sku], layer_height[sku], usable_rack_height)
    for sku in demand_8h
}


print(pallets_8h)
print(pallets_8h['0060972051'])

total_layers_required = {
    sku: math.ceil(demand_8h[sku] / cartons_per_layer[sku])  # Ceiling division for layers
    for sku in demand_8h
}

print(total_layers_required['0060972051'])

{'0015538304': 16, '006097205': 4, '0060972051': 4, '006097300A': 6, '006097400A': 4, '006097402': 4, '006408580E': 4, '006408581': 8, '006408585F': 4, '007237310': 16, '007290500D': 4, '007357357': 16, '007357534': 16, '007403110E': 4, '007403112E': 4, '007403113A': 4, '007403115D': 4, '007403118A': 4, '007403119A': 4, '007403119B': 4, '007403120B': 16, '007403122B': 16, '007403123': 16, '007403130A': 16, '007403133C': 16, '007403134': 4, '007403140A': 16, '007403153A': 8, '007403156A': 4, '007403158': 8, '007403160B': 4, '007403171A': 4, '007403174E': 4, '007403176A': 4, '007403183B': 16, '007403186': 16, '007403201A': 16, '007403206B': 16, '007403208A': 16, '007403222': 16, '007403230': 16, '007403259': 2, '007403300B': 6, '007403320B': 6, '007403400B': 6, '007403400BG': 6, '007403801': 4, '007403900': 16, '007483113A': 4, '007483300A': 4, '007517001': 16, '007517002': 16, '007525210E': 4, '007525435': 4, '007525535': 8, '007544540F': 6, '007544555': 6, '007544560': 6, '007574521B':

## Step 3 - Gurobi

In [3]:
# Create Gurobi Model
import gurobipy as gp
from gurobipy import GRB 

gurobi_params = {
    'WLSACCESSID':'a773530f-1497-481e-8314-843f89a61d1d',
    'WLSSECRET': '77535c46-5553-4eff-8dae-407de525be9d',
    'LICENSEID': 2568200
}
env = gp.Env(params=gurobi_params)
env.start()

model = gp.Model("Extended_Warehouse_Location_Problem", env=env)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2568200


GurobiError: License 2568200 has expired

## Step 4 - DV


In [ ]:

# Decision variables
x = model.addVars(material_demand['Raw Material No.'], range(total_racks), vtype=GRB.BINARY, name="x")  # SKU allocation to racks
y_r = model.addVars(total_racks, vtype=GRB.BINARY, name="y_r")  # Rack usage indicator
y_s = model.addVars(total_shelves, vtype=GRB.BINARY, name="y_s")  # Shelf usage indicator


# Objective: Minimize the number of shelves used
model.setObjective(gp.quicksum(y_r[r] for r in range(total_racks)), GRB.MINIMIZE)

# Constraints

# Constraint 1: Material Demand Satisfaction
for sku in pallets_8h:
    total_pallets_needed = pallets_8h[sku][0]  # Extract the total pallets required for the SKU
    model.addConstr(
        gp.quicksum(x[sku, r] for r in range(total_racks)) == total_pallets_needed,
        name=f"Demand_{sku}_5h"
    )
# 2. Rack Capacity (each rack holds up to 2 pallets)
for r in range(total_racks):
    model.addConstr(gp.quicksum(x[sku, r] for sku in pallets_8h) <= pallets_per_rack, name=f"Rack_Capacity_{r}")

# 3. Rack Height Constraint
for r in range(total_racks):
    usable_height = 1200 if (r % 5 != 4) else 300
    model.addConstr(
        gp.quicksum(x[sku, r] * layer_height[sku] for sku in pallets_8h) <= usable_height * y_r[r],
        name=f"Height_Constraint_Rack_{r}"
    )

# 4. Shelf Capacity (link racks to shelves)
for s in range(total_shelves):
    racks_in_shelf = range(s * racks_per_shelf, (s + 1) * racks_per_shelf)
    model.addConstr(
        gp.quicksum(x[sku, r] for sku in pallets_8h for r in racks_in_shelf) <= pallets_per_rack * racks_per_shelf * y_s[s],
        name=f"Shelf_Capacity_{s}"
    )

# 5. Rack-to-Shelf Mapping (each shelf must have exactly 5 active racks)
for s in range(total_shelves):
    racks_in_shelf = range(s * racks_per_shelf, (s + 1) * racks_per_shelf)
    model.addConstr(
        gp.quicksum(y_r[r] for r in racks_in_shelf) == racks_per_shelf * y_s[s],
        name=f"Rack_to_Shelf_Mapping_{s}"
    )


# 6. Total Pallets Constraint
model.addConstr(
    gp.quicksum(x[sku, r] for sku in pallets_8h for r in range(total_racks)) <= total_pallets,
    name="Max_Pallets"
)

# 7. Ensure each rack stores at most 2 SKUs (one SKU per pallet)
for r in range(total_racks):
    model.addConstr(
        gp.quicksum(x[sku, r] for sku in pallets_8h) <= pallets_per_rack,
        name=f"One_SKU_Per_Pallet_{r}"
    )

# Solve the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Number of Shelves Used: {model.ObjVal}")
    print(f"Total Pallets Used: {gp.quicksum(x[sku, r].X for sku in pallets_8h for r in range(total_racks))}")
    for s in range(total_shelves):
        if y_s[s].X > 0.5:
            print(f"Shelf {s} is used:")
            racks_in_shelf = range(s * racks_per_shelf, (s + 1) * racks_per_shelf)
            for r in racks_in_shelf:
                if y_r[r].X > 0.5:
                    print(f"  Rack {r} is active.")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.1.0 21A559)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Academic license 2568200 - for non-commercial use only - registered to bi___@nottingham.edu.cn
Optimize a model with 1515 rows, 113820 columns and 681240 nonzeros
Model fingerprint: 0x5fd9efba
Variable types: 0 continuous, 113820 integer (113820 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 422 rows and 770 columns
Presolve time: 1.01s
Presolved: 1093 rows, 113050 columns, 452340 nonzeros
Variable types: 0 continuous, 113050 integer (113050 binary)
Found heuristic solution: objective 185.0000000
Found heuristic solution: objective 180.0000000

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 

## Step 5 - Objective Function


In [ ]:
# Handle tuples and non-numeric values
total_demanded_pallets = sum(
    int(value[0]) if isinstance(value, tuple) else int(value)
    for value in pallets_8h.values()
)

# Print the result
print(f"Total Demanded Pallets: {total_demanded_pallets}")

Total Demanded Pallets: 347


In [ ]:
# Initialize dictionaries to store the assigned pallets per SKU
assigned_pallets = {sku: 0 for sku in pallets_8h}

# Calculate the assigned pallets from the model results
for sku in pallets_8h:
    assigned_pallets[sku] = sum(x[sku, r].X for r in range(total_racks))

# Calculate the total demanded pallets for comparison
demanded_pallets = {sku: pallets_8h[sku][0] for sku in pallets_8h}  # Extract demanded pallets

# Compare assigned pallets to demanded pallets
comparison_data = {
    "SKU": [],
    "Demanded Pallets": [],
    "Assigned Pallets": [],
    "Difference": []
}

for sku in pallets_8h:
    comparison_data["SKU"].append(sku)
    comparison_data["Demanded Pallets"].append(demanded_pallets[sku])
    comparison_data["Assigned Pallets"].append(assigned_pallets[sku])
    comparison_data["Difference"].append(assigned_pallets[sku] - demanded_pallets[sku])

# Convert to a DataFrame for better visualization
import pandas as pd
comparison_df = pd.DataFrame(comparison_data)

# Export to Excel
comparison_df.to_excel("pallets_comparison.xlsx", index=False)

# Print comparison summary
total_demanded = sum(demanded_pallets.values())
total_assigned = sum(assigned_pallets.values())
print(f"Total Demanded Pallets: {total_demanded}")
print(f"Total Assigned Pallets: {total_assigned}")
if total_demanded == total_assigned:
    print("All demanded pallets have been correctly assigned.")
else:
    print(f"Difference Detected: {total_assigned - total_demanded}")

# Preview the comparison DataFrame
print(comparison_df)

Total Demanded Pallets: 347
Total Assigned Pallets: 347.0
All demanded pallets have been correctly assigned.
            SKU  Demanded Pallets  Assigned Pallets  Difference
0    0015538304                 1               1.0         0.0
1     006097205                 1               1.0         0.0
2    0060972051                 1               1.0         0.0
3    006097300A                 1               1.0         0.0
4    006097400A                 1               1.0         0.0
..          ...               ...               ...         ...
319   85493001D                 1               1.0         0.0
320   85797001D                 1               1.0         0.0
321   85798001C                 1               1.0         0.0
322    95602003                 1               1.0         0.0
323    95953003                 1               1.0         0.0

[324 rows x 4 columns]


In [ ]:
# Calculate the total number of active racks
total_active_racks = sum(1 for r in range(total_racks) if y_r[r].X > 0.5)

# Calculate the total number of active shelves
total_active_shelves = sum(1 for s in range(total_shelves) if y_s[s].X > 0.5)

# Print the results
print(f"Total Active Racks Used: {total_active_racks}")
print(f"Total Active Shelves Used: {total_active_shelves}")

# Optional: Export to Excel for reporting
import pandas as pd

rack_shelf_data = {
    "Metric": ["Active Racks", "Active Shelves"],
    "Count": [total_active_racks, total_active_shelves]
}

rack_shelf_df = pd.DataFrame(rack_shelf_data)
rack_shelf_df.to_excel("rack_shelf_summary.xlsx", index=False)

print("Rack and Shelf usage summary exported to 'rack_shelf_summary.xlsx'.")


Total Active Racks Used: 175
Total Active Shelves Used: 35
Rack and Shelf usage summary exported to 'rack_shelf_summary.xlsx'.


## Step 6 - Constraints

Belum ngitung kalau ada lemari yang 300 cm 


In [ ]:
# After optimization, we'll extract and display the results

# Create a dictionary to store which SKU is assigned to each rack
assigned_skus = {r: [] for r in range(total_racks)}

# Assign SKUs to racks based on optimization results
for sku in demand_8h:
    for r in range(total_racks):
        if x[sku, r].X > 0.5:  # If the SKU is assigned to rack r
            assigned_skus[r].append(sku)

# Now, we want to focus on the racks that are top racks (where r % 5 == 4)
top_racks_info = []

for r in range(total_racks):
    # Check if this is a top rack (5th-level rack, index starts from 0, so r % 5 == 4)
    if r % 5 == 4:
        for sku in assigned_skus[r]:
            # Get the layer height for the SKU assigned to this rack
            sku_layer_height = layer_height[sku]
            
            # Usable space for the 5th-level racks is 300mm
            usable_space = 300
            
            # Compare layer height with usable space
            comparison_result = "OK" if sku_layer_height <= usable_space else "Exceeds"
            
            # Append information to the list
            top_racks_info.append({
                "Rack": r,
                "SKU": sku,
                "Pallet Height (Layer Height)": sku_layer_height,
                "Usable Space (300mm)": usable_space,
                "Comparison": comparison_result
            })

# Convert the top rack information to a pandas DataFrame for better display and export
import pandas as pd

top_racks_df = pd.DataFrame(top_racks_info)

# Print the result (You can also export it to Excel)
print(top_racks_df)

# Export to Excel (optional)
top_racks_df.to_excel("top_rack_sku_comparison.xlsx", index=False)

print("Top rack SKU assignment and comparison data has been exported to 'top_rack_sku_comparison.xlsx'.")




    Rack          SKU  Pallet Height (Layer Height)  Usable Space (300mm)  \
0     29   007403115D                           150                   300   
1     29   008077311D                           150                   300   
2     44   008140213B                           150                   300   
3     44   008140214A                           150                   300   
4     54   008173422A                           150                   300   
..   ...          ...                           ...                   ...   
62   324  5724010811F                           150                   300   
63   329   5436014713                           150                   300   
64   329    85492101B                           150                   300   
65   344   008230242D                           150                   300   
66   344   5436012113                           150                   300   

   Comparison  
0          OK  
1          OK  
2          OK  
3          

## Step 7 - Run

In [ ]:
# Initialize a counter for top racks used
top_racks_used = 0

# Loop through each rack
for r in range(total_racks):
    # Check if the rack is a top rack (5th-level, index starts from 0, so r % 5 == 4)
    if r % 5 == 4:
        # Check if any SKU is assigned to this top rack
        for sku in assigned_skus[r]:
            if x[sku, r].X > 0.5:  # If SKU is assigned to this rack
                top_racks_used += 1
                break  # No need to check further for this rack, it's already counted as used

# Print the total number of top racks used
print(f"Total Top Racks Used: {top_racks_used}")


Total Top Racks Used: 35
